## **Bienvenido a la tercera práctica de Jupyter Notebook**

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jpcano1/MINE-4101-tareas/blob/master/lab_3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

## **Caso**
- Teniendo en cuenta los datos recopilados de la encuesta Nacional de uso del tiempo-ENUT¹, utilizados en la tarea anterior:
    1. Plantea las hipótesis de negocio que permitan profundizar el objetivo principal de ENUT (ver sección 1.2 objetivos del Manual de conceptos básicos ENUT²).
    2. Identifique las variables que considere necesarias para dar respuesta a estas hipótesis y construya el respectivo dataframe. Puede utilizar todas las fuentes disponibles que se encuentran en la página del DANE
    3. Realice un estudio de calidad de datos sobre el dataframe consolidado en el punto anterior, para ello, utilice los conceptos vistos en clase (ejemplo: completitud).
    4. Realice un análisis estadístico univariado que permita validar las hipótesis planteadas en el punto 1 y presente las respectivas conclusiones.
    
> ¹http://microdatos.dane.gov.co/index.php/catalog/214/get_microdata

> ²http://microdatos.dane.gov.co/index.php/catalog/214/related_materials

***

- **Objetivo General**:

La encuesta Nacional del Uso del Tiempo, es una investigación desarrollada por el DANE, con el fin de generar información sobre el tiempo dedicado por la   población de 10 años y más a actividades de trabajo y personales

- **Objetivos específicos**:
    1. Proporcionar información para la construcción de la cuenta satélite de trabajo no remunerado.
    2. Producir información para la formulación, seguimiento y evaluación de la política pública con enfoque de género.
    3. Proporcionar información para la formulación, seguimiento y evaluación de la política pública en temas relacionados con el uso del tiempo.

## **1. Hipótesis de Trabajo**
- El estrato influye en el número de horas trabajadas a la semana
- Dependiendo del estrato, se sabrá si el usuario vive con el papá o la mamá
- Dependiendo de la región del país, se tendrá o no un sistema de acueducto.
- (Plan B) Dependiendo del estrato se sabrá si el usuario cuenta con un trabajo secundario.

## **Indentificación de Variables**

In [ ]:
!pip install -q pyreadstat
!pip install -q https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [12]:
import pandas as pd
import numpy as np

import requests

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import pandas_profiling

### **Obtención de Datos**

In [14]:
# condiciones_vivienda.sav
url_condiciones = "https://github.com/jpcano1/MINE-4101-tareas/blob/master/\
datasets/CAPITULO%20B.%20CONDICIONES%20DE%20LA%20VIVIENDA.sav?raw=true" 

# fuerza_trabajo.sav
url_fuerza = "https://github.com/jpcano1/MINE-4101-tareas/blob/master/\
datasets/CAPITULO%20H.%20FUERZA%20DE%20TRABAJO%20-%20Uso%20del%20tiempo%20e%20ingresos.sav?raw=true" 

# composicion_hogar.sav
url_composicion = "https://github.com/jpcano1/MINE-4101-tareas/blob/master/\
datasets/CAPITULO%20D.%20COMPOSICION%20DEL%20HOGAR.sav?raw=true" 

In [35]:
def null_percentage(df, int_mode=False):
    """
    :param df:
    :param int_mode:
    :return:
    """
    null_percentages = dict()
    for col in df.columns:
        percentage = df[col].isna().sum() * 100
        percentage /= df.shape[0]
        if not int_mode:
            percentage = f"{percentage}% of null values"
        null_percentages[col] = percentage
    return null_percentages

def download_content(url, filename="image.jpg"):
    """
    Función que se encarga de descargar un archivo deseado
    :param url: la url de descarga
    :param filename: El nombre del archivo
    """
    r = requests.get(url)
    with open(filename, "wb") as f:
        f.write(r.content)
    return

def drop_null_column(df, drop_perc=85):
    """
    :param df:
    :return:
    """
    null_perc = null_percentage(df, int_mode=True)
    for col in df.columns:
        if null_perc[col] > drop_perc:
            df.drop(columns=col, inplace=True)

In [13]:
download_content(url_condiciones, "condiciones_vivienda.sav")
download_content(url_fuerza, "fuerza_trabajo.sav")
download_content(url_composicion, "composicion_hogar.sav")

In [38]:
data_condiciones = pd.read_spss("condiciones_vivienda.sav")
data_fuerza = pd.read_spss("fuerza_trabajo.sav")
data_composicion = pd.read_spss("composicion_hogar.sav")

In [17]:
columns_condiciones = {
    "FEX_C": "factores_expansion",
    "P424": "tipo_vivienda",
    "P4030S1": "energia_electrica",
    "P4030S1A1": "estrato",
    "P4030S5": "acueducto",
    "P4030S3": "alcantarillado",
    "P4030S4": "recoleccion_basuras",
    "P4030S4A1": "dias_x_semana",
    "P4030S2": "gas_natural_dom",
    "P4020": "material_pisos",
    "DIA_REFERENCIA_2": "dia_referencia"
}

In [25]:
columns_fuerza = {
    'P1099S1': 'horas_ida',
    'P1099S2': 'minutos_ida',
    'P1145': 'ganancia_ayudas',
    'P1150': 'trabajo_dia',
    'P1150S1': 'horas_trabajo',
    'P1150S2': 'minutos_trabajo',
    'P1151': 'horas_semana',
    'P427': 'este_trabajo',
    'P428': 'numero_personas',
    'P6250': 'empleo_secundario',
    'P6370S1': 'oficio_princ',
    'P6390S1': 'rama_princ',
    'P6400': 'misma_empresa',
    'P6426': 'cantidad_tiempo',
    'P6440': 'contrato',
    'P6500': 'ganancias_mes_pasado',
    'P6510': 'ganancia_jubilacion',
    'P6545': 'ganancia_arriendo',
    'P6750': 'ganancia_neta_mes_pasado',
    'P6760': 'numero_meses',
    'P6880': 'lugar_trabajo'
 }

In [43]:
columns_composicion = {
    'P1172': 'vive_conyuge',
    'P1172S1': 'numero_orden',
    'P1173': 'se_reconoce',
    'P1174': 'comparte_jefatura',
    'P425': 'parentesco_jefe',
    'P426': 'estado_civil',
    'P5754': 'vive_con_padre',
    'P5754S2': 'numero_orden_padre',
    'P5762': 'vive_con_madre',
    'P5762S2': 'numero_orden_madre',
    'P6020': 'sexo',
    'P6040': 'edad'
 }

### **Limpieza de datos**

In [40]:
drop_null_column(data_condiciones)
drop_null_column(data_fuerza)
drop_null_column(data_composicion)

In [44]:
data_condiciones.rename(columns=columns_condiciones, inplace=True)
data_condiciones.rename(columns=lambda x: x.lower(), inplace=True)

data_fuerza.rename(columns=columns_fuerza, inplace=True)
data_fuerza.rename(columns=lambda x: x.lower(), inplace=True)

data_composicion.rename(columns=columns_composicion, inplace=True)
data_composicion.rename(columns=lambda x: x.lower(), inplace=True)

In [48]:
variables_fuerza = [
    "directorio", "contrato", "horas_semana", 
    "empleo_secundario"
]

variables_condiciones = [
    "directorio",  "estrato", "region", "acueducto",
    "alcantarillado", "gas_natural_dom"
]

variables_composicion = [
    "directorio", "vive_con_padre", 
    "vive_con_madre"
]

In [49]:
data_condiciones = data_condiciones[variables_condiciones]
data_fuerza = data_fuerza[variables_fuerza]
data_composicion = data_composicion[variables_composicion]